This file priviately reproduces vbf-filtered DY in UL setting and so that we could later compare it with those produced from GEN-SIM from the RERECO sample

## NOTE:
for whatever reason, the environement we need doesn't work well with jupyter notebook, so please use the bash commands below from a separate terminal (from start to finish)

Download the fragment file 

In [2]:
%%bash
curl -s -k https://cms-pdmv-prod.web.cern.ch/mcm/public/restapi/requests/get_fragment/HIG-RunIIFall18wmLHEGS-00419 --retry 3 --create-dirs -o Configuration/GenProduction/python/HIG-RunIIFall18wmLHEGS-00419-fragment.py
[ -s Configuration/GenProduction/python/HIG-RunIIFall18wmLHEGS-00419-fragment.py ] || exit $?;

Install CMSSW (we assume voms-proxy-init -voms cms is done)

In [ ]:
%%bash
export SCRAM_ARCH=slc7_amd64_gcc700
source /cvmfs/cms.cern.ch/cmsset_default.sh

cmsrel CMSSW_10_6_17_patch1
cd CMSSW_10_6_17_patch1/src

eval `scram runtime -sh`
mv ../../Configuration .
scram b -j8
cd ../..


Step 1 Gen Sim

In [ ]:
%%bash

cmsDriver.py Configuration/GenProduction/python/HIG-RunIIFall18wmLHEGS-00419-fragment.py \
    --python_filename HIG-RunIIFall18wmLHEGS-00419_cfg.py \
    --eventcontent RAWSIM \
    --customise Configuration/DataProcessing/Utils.addMonitoring \
    --datatier GEN-SIM \
    --fileout file:HIG-RunIIFall18wmLHEGS.root \
    --conditions 106X_upgrade2018_realistic_v4 \
    --beamspot Realistic25ns13TeVEarly2018Collision \
    --customise_commands process.source.numberEventsInLuminosityBlock="cms.untracked.uint32(250)" \
    --step LHE,GEN,SIM \
    --geometry DB:Extended \
    --era Run2_2018 \
    --no_exec \
    --mc \
    -n 1000

cmsRun HIG-RunIIFall18wmLHEGS-00419_cfg.py


Step 2 Digi

In [ ]:
%%bash

nohup cmsDriver.py  \
    --python_filename HIG-RunIIFall18wmDIGI_cfg.py \
    --eventcontent RAWSIM \
    --pileup 2018_25ns_UltraLegacy_PoissonOOTPU \
    --customise Configuration/DataProcessing/Utils.addMonitoring \
    --datatier GEN-SIM-DIGI \
    --fileout file:HIG-RunIIFall18wmDIGI.root \
    --pileup_input "dbs:/MinBias_TuneCP5_13TeV-pythia8/RunIISummer20UL18SIM-106X_upgrade2018_realistic_v11_L1v1-v2/GEN-SIM" \
    --conditions 106X_upgrade2018_realistic_v11_L1v1 \
    --step DIGI,L1,DIGI2RAW \
    --geometry DB:Extended \
    --filein file:HIG-RunIIFall18wmLHEGS.root \
    --era Run2_2018 \
    --runUnscheduled \
    --no_exec \
    --mc \
    -n 1000
cmsRun HIG-RunIIFall18wmDIGI_cfg.py &> Step2Digi_output.log

Step 2.5 HLT (stop here and redo all the previous steps with -n 1000 bc this step requires a seperate and new env (idk why)